# Implementing DLT Pipelines

FUNKTIONIERT NICHT IN DER FREE EDITION!


<div  style="text-align: center;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/oreilly-databricks-dea/main/Includes/Images/school_schema.png" alt="School Schema">
</div>

In [0]:
SET school.dataset_path=dbfs:/mnt/DE-Associate-Book/datasets/school;

## Bronze layer

#### Creating a streaming table

In [0]:
-- Deklaration einer DLT-Tabelle in SQL, um inkrementelle Daten aus der Datenquelle (mit Auto Loader) zu lesen
CREATE OR REFRESH STREAMING TABLE enrollments_raw -- Das Schlüsselwort STREAMING wird verwendet, da die Quelle inkrementelle Daten liefert.
COMMENT "The raw courses enrollments, ingested from enrollments-dlt-raw folder" -- Fügt der Tabelle eine beschreibende Anmerkung für zusätzlichen Kontext hinzu.
AS SELECT * FROM cloud_files( -- Die Methode cloud_files ist der direkte Weg, um Auto Loader in SQL zu verwenden.
                            "${school.dataset_path}/enrollments-dlt-raw", -- 1. Der Quellspeicherort der Dateien.
                            "json", -- 2. Das Dateiformat.
                            map("cloudFiles.inferColumnTypes", "true") -- 3. Eine Map mit Lese-Einstellungen. Diese Option weist Auto Loader an, die Spaltentypen automatisch zu erkennen.
                           )

#### Creating a materialized view

In [0]:
-- Erstellt eine Materialized View, die als Lookup-Tabelle für Studentendaten dient.
CREATE OR REPLACE MATERIALIZED VIEW students
COMMENT "The students lookup table, ingested from students-json"
-- Eine Materialized View wird verwendet, da die Quelldaten Updates oder Deletes enthalten können,
-- was dem "Append-Only"-Prinzip von Streaming-Tabellen widerspricht.
AS SELECT * FROM json.`${school.dataset_path}/students-json`
-- Bei jedem Pipeline-Lauf wird das gesamte Quell-Dataset neu verarbeitet,
-- um sicherzustellen, dass alle Änderungen in den Daten erfasst werden.


## Silver layer

In [0]:
-- Erstellt eine Streaming-Tabelle für die Silver-Schicht, um Rohdaten zu bereinigen und anzureichern.
-- Die Zieltabelle wird als STREAMING TABLE deklariert, weil die Quelle ebenfalls als Stream gelesen wird.
CREATE OR REFRESH STREAMING TABLE enrollments_cleaned (
  -- Eine DLT Expectation wird als Qualitätskontrolle mittels CONSTRAINT hinzugefügt.
  -- Datensätze, die diese Bedingung verletzen (enroll_id ist NULL), werden automatisch verworfen (DROP ROW).
  CONSTRAINT valid_order_number EXPECT (enroll_id IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "The cleaned courses enrollments with valid enroll_id"
AS
 SELECT enroll_id, quantity, o.student_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
        cast(from_unixtime(enroll_timestamp, 'yyyy-MM-dd HH:mm:ss') AS timestamp) formatted_timestamp,
        o.courses, c.profile:address:country as country
 -- Die STREAM-Methode liest die Tabelle als inkrementelle Streaming-Quelle.
 -- Die LIVE-Syntax dient als Namespace, um andere DLT-Objekte in derselben Pipeline zu referenzieren.
 FROM STREAM(LIVE.enrollments_raw) o
 -- Die Streaming-Daten werden durch einen Join mit der Studententabelle angereichert.
 LEFT JOIN LIVE.students c
   ON o.student_id = c.student_id



## Gold layer

In [0]:
-- Erstellt ein Gold-Objekt, das die finale und am höchsten veredelte Stufe der Pipeline darstellt.
-- Die Tabelle wird als Materialized View deklariert, damit die aggregierten Werte bei jedem Pipeline-Update ersetzt werden.
CREATE OR REPLACE MATERIALIZED VIEW uk_daily_student_courses
COMMENT "Daily number of courses per student in United Kingdom"
AS
 -- Die Abfrage erstellt ein hoch aggregiertes, für Analysen optimiertes Format.
 -- Sie berechnet die tägliche Anzahl der Kurse pro Student.
 SELECT student_id, f_name, l_name, date_trunc("DD", formatted_timestamp) order_date, sum(quantity) courses_counts
 FROM LIVE.enrollments_cleaned
 -- Die Daten werden für eine spezifische Region ("United Kingdom") gefiltert.
 WHERE country = "United Kingdom"
 GROUP BY student_id, f_name, l_name, date_trunc("DD", formatted_timestamp)

# Configuring DLT Pipelines

## Modifying DLT pipelines

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW fr_daily_student_courses
COMMENT "Daily number of courses per student in France"
AS
 SELECT student_id, f_name, l_name, date_trunc("DD", formatted_timestamp) order_date, sum(quantity) courses_counts
 -- FROM enrollments_cleaned
 FROM LIVE.enrollments_cleaned
 WHERE country = "France"
 GROUP BY student_id, f_name, l_name, date_trunc("DD", formatted_timestamp)